In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Linear methods for non-linear data

In [ ]:
# Generate non-linear data
np.random.seed(1)
x = np.arange(-1, 1, 0.1).reshape(-1, 1)
y = 0.5*x + 2*x**2

%matplotlib notebook

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.scatter(x, y, c='r')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
plt.show()

In [ ]:
# Fit linear regression
lr = LinearRegression()
lr.fit(x, y)

# Plot the regression line above
y_pred = lr.predict(x)
ax1.plot(x, y_pred)
plt.show()

# Nonlinear transform

Transform data to 2-dimensional space using polynomial basis functions: $$ \phi(x) = (x, x^2)$$

In [ ]:
x2 = x**2

%matplotlib notebook

# Plot the transformed points 
fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')
ax2.scatter(x, x2, y, c='r', linewidths=5, alpha=1)
ax2.set_xlabel('x')
ax2.set_ylabel('x^2')
ax2.set_zlabel('y')
plt.show()

The transformed data can be modeled using a linear model in a higher dimensional space

In [ ]:
# Plot a plane through to transformed points
p = np.linspace(-1, 1, 21)

xx, xx2 = np.meshgrid(p, p)

z = 0.5*xx + 2*xx2

ax2.plot_surface(xx, xx2, z)
plt.show()

# Non-linear regression with basis functions

In [ ]:
# Data-generating function
def f(x):
    #return x**2 - 4*x  # Feel free to change the formula to test different kinds of functions
    return np.sin(2*x) - 3*np.sin(x) + 0.05*x**2
    #return 2*x
    
# Returns a noisy sample from f
def noisyF(x, sigma=0.1):
    return np.random.normal(f(x), sigma)
    
# Samples n_samples independent noisy samples from f where x is selected uniformly at random from interval
def sampleF(n_samples, interval=[0, 6], sigma=0.1):
    x = np.random.uniform(0, 6, n_samples)
    y = noisyF(x, sigma)
    return x, y

In [ ]:
# Generate data
interval = [0,6]
sigma = 0.8

np.random.seed(1)
n_samples = 20
x_train, y_train = sampleF(n_samples, interval=interval, sigma=sigma)

In [ ]:
# Plot data
%matplotlib inline

plt.scatter(x_train, y_train, c='r')
plt.show()

In [ ]:
# Define the basis functions
def phi(x):
    n_samples = len(x)
    #return np.matrix([np.ones(n_samples), x, np.sin(x), np.cos(x), np.cos(2*x), np.sin(2*x), np.cos(3*x), np.sin(3*x)]).T
    #return np.matrix([np.ones(n_samples), x]).T
    #return np.matrix([np.ones(n_samples), x, x**2]).T
    #return np.matrix([np.ones(n_samples), x, x**2, x**3]).T
    #return np.matrix([np.ones(n_samples), x, x**2, x**3, x**4]).T
    return np.matrix([np.ones(n_samples), x, x**2, x**3, x**4, x**5, x**6, x**7, x**8, x**9]).T

In [ ]:
# Print original data
x_train

In [ ]:
# Transform data to high dimensional space
X = phi(x_train)
Y = np.matrix(y_train).T
X

In [ ]:
# Multiple the design matrix X with its transpose
XTX = X.T*X
XTX

In [ ]:
# Inverse XTX
XTX_inv = np.linalg.inv(XTX)
XTX_inv

In [ ]:
# Check that the inverse is sensible
XTX*XTX_inv

In [ ]:
# Compute the numerator
XTy = X.T*Y
XTy

In [ ]:
# Solve the weights
w = XTX_inv*XTy
w

In [ ]:
# Plot the learned model
x_grid = np.linspace(0, 6, 121)
y_pred = w.T*phi(x_grid).T

plt.plot(x_grid, np.asarray(y_pred)[0], linewidth=2)
plt.scatter(x_train, y_train, c='red')
plt.show()

pred = (w.T*X.T).T
mse = mean_squared_error(pred, y_train)
print('Training error: %.3f' % mse)

In [ ]:
# Generate new samples and plot with the learned curve
np.random.seed(7)

x_val, y_val = sampleF(n_samples, interval=interval, sigma=sigma)

plt.plot(x_grid, np.asarray(y_pred)[0], linewidth=2)
plt.scatter(x_val, y_val, c='m')
plt.show()

pred = (w.T*phi(x_val).T).T
mse = mean_squared_error(pred, y_val)
print('Validation error: %.3f' % mse)

# Effect of regularization

In [ ]:
# L2 Regularization
l = 1 # lambda parameter specifying the strength of regularization
I = np.identity(X.shape[1])

w_reg = np.linalg.inv(XTX + l*I)*XTy # Weights with regularization
w_reg

In [ ]:
# Plot the regression lines
y_pred_reg = w_reg.T*phi(x_grid).T

plt.figure()

plt.scatter(x_train, y_train, c='red')
plt.plot(x_grid, np.asarray(y_pred)[0], linewidth=3)
plt.plot(x_grid, np.asarray(y_pred_reg)[0], c='g', linewidth=3)
plt.legend(['Unregularized', 'Regularized'])
plt.title('Training data')

plt.figure()

plt.scatter(x_val, y_val, c='red')
plt.plot(x_grid, np.asarray(y_pred)[0], linewidth=3)
plt.plot(x_grid, np.asarray(y_pred_reg)[0], c='g', linewidth=3)
plt.legend(['Unregularized', 'Regularized'])
plt.title('Validation data')

plt.show()

pred = (w_reg.T*phi(x_train).T).T
mse = mean_squared_error(pred, y_train)
print('Training error after regularization: %.3f' % mse)

pred = (w_reg.T*phi(x_val).T).T
mse = mean_squared_error(pred, y_val)
print('Validation error after regularization: %.3f' % mse)